In [1]:
import hopsworks
import great_expectations as ge
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

import math
import pandas as pd

from features.transactions import calculate_time_delta_t_minus_1, calculate_loc_delta_t_minus_1

In [2]:
# Connect to the Hopsworks Feature Store
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/15479
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
# Read the historical transaction data
transactions_pdf = pd.read_csv("../data_generation/historical_transactions.csv", parse_dates=['datetime'])

In [4]:
transactions_pdf = transactions_pdf[["datetime", "cc_num", "latitude", 'longitude']]

# cc_num is more of an account_id rather than a proper credit card number, rename the column to avoid confusion
transactions_pdf.rename(columns={'cc_num': 'account_id', 'datetime': 'last_transaction_datetime'}, inplace=True) 

In [5]:
profiles_last_transaction_pdf = transactions_pdf.sort_values('last_transaction_datetime').groupby('account_id').tail(1)

In [6]:
profiles_last_transaction_pdf

,last_transaction_datetime,account_id,latitude,longitude
46092,2024-05-30 14:54:28,77025ad95578f857bd6a7390b92e8682,39.717340,-74.969330
47607,2024-06-04 14:30:15,018ff07230cc505876224e941dfd6096,42.527870,-70.928660
48423,2024-06-07 08:02:31,7d772777e1409e7191e646e58811b444,39.334270,-76.439410
48710,2024-06-08 07:47:30,858eeba131592e1a20bba8a83290feb9,31.845680,-102.367640
62072,2024-06-08 20:12:56,46625b13e486eddd36653c86357b99e5,29.657426,-82.316532
...,...,...,...,...
53995,2024-06-25 22:45:14,71884f7fc735581e5151d76e61e22740,35.052660,-78.878360
53996,2024-06-25 22:48:04,3f6bf7522fe533c8ba0c802d14197bf2,33.352830,-111.789030
53997,2024-06-25 22:50:19,b27647b73dcc913bf07f58ab066e32e8,42.583420,-71.802300
53998,2024-06-25 22:53:33,0eea379415b6009b54f528c1f79a78f9,40.557600,-74.284590


In [7]:
# Create the feature group metadata
profiles_last_transaction_fg = fs.get_or_create_feature_group(
    name="profiles_last_transaction",
    version=1,
    description="Last transaction time and location for each account",
    primary_key=['account_id'],
    event_time='last_transaction_datetime',
    online_enabled=True,
    statistics_config={'histograms': True, 'correlations': True}
)

In [8]:
# Insert data into feature group
profiles_last_transaction_fg.insert(profiles_last_transaction_pdf)

Feature Group created successfully, explore it at 
https://snurran.hops.works/p/15479/fs/15427/fg/16399


Uploading Dataframe: 0.00% |          | Rows 0/1000 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: profiles_last_transaction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/15479/jobs/named/profiles_last_transaction_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7ff1cbbeff40>, None)